In [15]:
import numpy as np
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue
from random import random

In [16]:
PROBLEM_SIZE = 5
NUM_SETS = 10
SETS = tuple(
    np.array([random() < 0.3 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS)
)
print(SETS[0])
State = namedtuple("State", ["taken", "not_taken"])

[False False  True False False]


In [17]:
# Check if the state is the goal state
def goal_check(state):
    return np.all(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )

In [18]:
assert goal_check(State(set(range(NUM_SETS)), set())), "Problem not solvable"

In [19]:
frontier = PriorityQueue()
frontier.put(State(set(), set(range(NUM_SETS))))

counter = 0
current_state = frontier.get()
while not goal_check(current_state):
    counter += 1
    for action in current_state[1]:
        new_state = State(
            current_state.taken ^ {action}, current_state.not_taken ^ {action}
        )
        frontier.put(new_state)
    current_state = frontier.get()

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

Solved in 16 steps (3 tiles)


In [20]:
current_state

State(taken={9, 5, 6}, not_taken={0, 1, 2, 3, 4, 7, 8})

In [21]:
goal_check(current_state)

True

In [22]:
# This code will use the number of Falses in the
# resulting set as the distance to the solution,
# minimizing it

frontier = PriorityQueue()
frontier.put((0, State(set(), set(range(NUM_SETS)))))

counter = 0
_, current_state = frontier.get()

while not goal_check(current_state):
    counter += 1
    for action in current_state.not_taken:
        new_state = State(
            current_state.taken ^ {action}, current_state.not_taken ^ {action}
        )
        new_cost = PROBLEM_SIZE - np.count_nonzero(
            reduce(
                np.logical_or,
                [SETS[i] for i in new_state.taken],
                np.array([0] * PROBLEM_SIZE),
            )
        )

        frontier.put((new_cost, new_state))
    _, current_state = frontier.get()

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")
print(current_state)

Solved in 3 steps (3 tiles)
State(taken={1, 5, 9}, not_taken={0, 2, 3, 4, 6, 7, 8})


In [23]:
# Solve using A*
def cost(state):
    return len(state.taken)


def heuristic(state):
    return PROBLEM_SIZE - np.count_nonzero(
        reduce(
            np.logical_or,
            [SETS[i] for i in new_state.taken],
            np.array([0] * PROBLEM_SIZE),
        )
    )

# TODO: Build a new H, consider "special sets", consider order of the sets

frontier = PriorityQueue()
frontier.put((0, State(set(), set(range(NUM_SETS)))))

counter = 0
_, current_state = frontier.get()

while not goal_check(current_state):
    counter += 1
    for action in current_state.not_taken:
        new_state = State(
            current_state.taken ^ {action}, current_state.not_taken ^ {action}
        )
        new_cost = cost(new_state) + heuristic(new_state)
        frontier.put((new_cost, new_state))
    _, current_state = frontier.get()

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")
print(current_state)

Solved in 5 steps (3 tiles)
State(taken={8, 1, 9}, not_taken={0, 2, 3, 4, 5, 6, 7})
